In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set(style="whitegrid")

def load_data(db_name="vacancies.db"):
    conn = sqlite3.connect(db_name)
    query = "SELECT * FROM vacancies"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

data = load_data()
print(f"Размер данных: {data.shape}")
print(data.head())


Размер данных: (1994, 15)
          id                                               name  \
0  112836913                            Data Scientist (NLP&ML)   
1  112836663                                     Data Scientist   
2  112029463              Аналитик по искусственному интеллекту   
3  112835758                                     Data Scientist   
4  106818906  Инженер данных/Data Engineer в Управление анал...   

         area_name  salary_from  salary_to salary_currency  \
0           Москва          NaN        NaN            None   
1           Москва          NaN        NaN            None   
2  Нижний Новгород          NaN        NaN            None   
3           Москва          NaN        NaN            None   
4           Москва          NaN        NaN            None   

               published_at  \
0  2024-12-10T10:25:06+0300   
1  2024-12-10T11:06:46+0300   
2  2024-12-10T09:00:24+0300   
3  2024-12-10T10:11:16+0300   
4  2024-12-10T10:28:46+0300   

           

In [2]:
data["published_at"] = pd.to_datetime(data["published_at"], errors="coerce")
data["published_at"] = data["published_at"].dt.date
data = data.dropna(subset=['snippet_requirement', 'snippet_responsibility'])
data['salary_to'] = data['salary_to'].fillna(data['salary_from'])
data.head()

,id,name,area_name,salary_from,salary_to,salary_currency,published_at,employer_name,alternate_url,snippet_requirement,snippet_responsibility,professional_roles,schedule,employment,experience
0,112836913,Data Scientist (NLP&ML),Москва,NaN,NaN,None,2024-12-10,АКБ Национальный Резервный Банк,https://hh.ru/vacancy/112836913,Будет плюсом: — Окончание программы ШАД Яндекс...,Анализ новостей на финансовых рынках. — Задачи...,Дата-сайентист,Полный день,Полная занятость,От 1 года до 3 лет
1,112836663,Data Scientist,Москва,NaN,NaN,None,2024-12-10,VK,https://hh.ru/vacancy/112836663,Общая математическая культура (теория вероятно...,...методов машинного обучения и собственной пл...,Дата-сайентист,Полный день,Полная занятость,От 1 года до 3 лет
2,112029463,Аналитик по искусственному интеллекту,Нижний Новгород,NaN,NaN,None,2024-12-10,Региональный Центр Поддержки и Координации Оте...,https://hh.ru/vacancy/112029463,...Опыт взаимодействия с моделями <highlightte...,Оценка текущего состояния функции <highlightte...,Аналитик,Полный день,Полная занятость,От 1 года до 3 лет
3,112835758,Data Scientist,Москва,NaN,NaN,None,2024-12-10,ИК СИБИНТЕК,https://hh.ru/vacancy/112835758,Владение Python и классическим <highlighttext>...,Анализ <highlighttext>данных</highlighttext> д...,Дата-сайентист,Удаленная работа,Полная занятость,От 3 до 6 лет
4,106818906,Инженер данных/Data Engineer в Управление анал...,Москва,NaN,NaN,None,2024-12-10,Газпромбанк,https://hh.ru/vacancy/106818906,Обязательно знание SQL (разные варианты диалек...,...процессов поставки <highlighttext>данных</h...,Дата-сайентист,Удаленная работа,Полная занятость,От 1 года до 3 лет


In [3]:
# Проверка пропусков
missing_values = data.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_values / len(data)) * 100
print("Пропуски в данных:")
print(pd.DataFrame({"Пропущено": missing_values, "Процент": missing_percentage}))

# Типы данных
print("\nТипы данных:")
print(data.dtypes)


Пропуски в данных:
                        Пропущено    Процент
salary_from                  1417  71.601819
salary_to                    1327  67.054068
salary_currency              1327  67.054068
area_name                       0   0.000000
name                            0   0.000000
id                              0   0.000000
published_at                    0   0.000000
employer_name                   0   0.000000
alternate_url                   0   0.000000
snippet_requirement             0   0.000000
snippet_responsibility          0   0.000000
professional_roles              0   0.000000
schedule                        0   0.000000
employment                      0   0.000000
experience                      0   0.000000

Типы данных:
id                         object
name                       object
area_name                  object
salary_from               float64
salary_to                 float64
salary_currency            object
published_at               object
employer_

In [4]:
import pandas as pd
import re
from keybert import KeyBERT
from spacy.lang.ru.stop_words import STOP_WORDS as RUSSIAN_STOP_WORDS
from tqdm import tqdm
import spacy

warnings.filterwarnings("ignore")

# Загружаем модель spaCy для русского языка
nlp = spacy.load('ru_core_news_sm')

# Функция для очистки текста
def clean_text(text):
    if pd.isnull(text):
        return text
    text = re.sub(r'<[^>]+>', '', text)  # Удаляем HTML-теги
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы
    return text

# Функция для лемматизации текста
def lemmatize_text(text):
    doc = nlp(text)
    # Лемматизируем все токены и объединяем их в строку
    return ' '.join([token.lemma_ for token in doc])

# Очистка данных
data['snippet_requirement'] = data['snippet_requirement'].apply(clean_text)
data['snippet_responsibility'] = data['snippet_responsibility'].apply(clean_text)

# Лемматизация текста перед анализом
data['snippet_requirement'] = data['snippet_requirement'].apply(lemmatize_text)
data['snippet_responsibility'] = data['snippet_responsibility'].apply(lemmatize_text)

# Инициализация модели KeyBERT
model = KeyBERT(model='DeepPavlov/rubert-base-cased')

# Функция для извлечения ключевых слов с использованием KeyBERT
def extract_keybert_keywords(text, ngram_range=(1, 3), top_n=20):
    if pd.isnull(text) or len(text.strip()) == 0:
        return ''
    keywords = model.extract_keywords(
        text,
        keyphrase_ngram_range=ngram_range,  # Диапазон n-грамм
        stop_words=list(RUSSIAN_STOP_WORDS),  # Стоп-слова для русского языка
        top_n=top_n  # Количество ключевых слов/словосочетаний
    )
    return ', '.join([kw[0] for kw in keywords])

# Использование tqdm для отображения прогресса
tqdm.pandas(desc="Extracting keywords for requirements")
data['keywords_requirement_keybert'] = data['snippet_requirement'].progress_apply(
    lambda x: extract_keybert_keywords(x)
)

tqdm.pandas(desc="Extracting keywords for responsibilities")
data['keywords_responsibility_keybert'] = data['snippet_responsibility'].progress_apply(
    lambda x: extract_keybert_keywords(x)
)


No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with mean pooling.
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [5]:
data.head()

,id,name,area_name,salary_from,salary_to,salary_currency,published_at,employer_name,alternate_url,snippet_requirement,snippet_responsibility,professional_roles,schedule,employment,experience,keywords_requirement_keybert,keywords_responsibility_keybert
0,112836913,Data Scientist (NLP&ML),Москва,NaN,NaN,None,2024-12-10,АКБ Национальный Резервный Банк,https://hh.ru/vacancy/112836913,быть плюс : — окончание программа шад яндекс (...,анализ новость на финансовый рынок . — задача ...,Дата-сайентист,Полный день,Полная занятость,От 1 года до 3 лет,"prod ускорение инференса, инференса nlp модель...","recognition topic modelling, entity recognitio..."
1,112836663,Data Scientist,Москва,NaN,NaN,None,2024-12-10,VK,https://hh.ru/vacancy/112836663,общий математический культура ( теория вероятн...,... метод машинный обучение и собственный плат...,Дата-сайентист,Полный день,Полная занятость,От 1 года до 3 лет,"культура теория вероятность, вероятность матем...","модель основный область, метод машинный обучен..."
2,112029463,Аналитик по искусственному интеллекту,Нижний Новгород,NaN,NaN,None,2024-12-10,Региональный Центр Поддержки и Координации Оте...,https://hh.ru/vacancy/112029463,... опыт взаимодействие с модель искусственный...,оценка текущий состояние функция данных и анал...,Аналитик,Полный день,Полная занятость,От 1 года до 3 лет,"обучение наличие опыт, интеллект машинный обуч...","состояние функция аналитика, функция аналитика..."
3,112835758,Data Scientist,Москва,NaN,NaN,None,2024-12-10,ИК СИБИНТЕК,https://hh.ru/vacancy/112835758,владение python и классический ml - стэком ( p...,"анализ данных для моделирование ( подготовка ,...",Дата-сайентист,Удаленная работа,Полная занятость,От 3 до 6 лет,"scikit learn xgboost, python классический ml, ...","формирование выборка моделирование, проработка..."
4,106818906,Инженер данных/Data Engineer в Управление анал...,Москва,NaN,NaN,None,2024-12-10,Газпромбанк,https://hh.ru/vacancy/106818906,Обязательно знание sql ( разный вариант диалек...,... процесс поставка данных для продуктовый ко...,Дата-сайентист,Удаленная работа,Полная занятость,От 1 года до 3 лет,"знание sql разный, обязательно знание sql, зна...","команда data scientist, data scientist ов, ana..."
